In [1]:
import pandas as pd
import seaborn as sns
import time
import re
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [9]:
url = "https://nec.gov.np/applicant/cat_online/34"
driver = webdriver.Chrome()
driver.get(url)

element = driver.find_element("id", "datatables1_info")
text = element.text
numbers = re.findall(r'\d+', text)
numbers = list(map(int, numbers))

total_entries = numbers[2]
grouping_of = numbers[1]
no_pages = int(np.ceil(total_entries/grouping_of))
print(no_pages)

titles = []


for page in range(1, no_pages+1):
    # Wait for the table to load
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))
    
    # Extract data from the current page
    rows = driver.find_elements(By.TAG_NAME, "tr")[1:]  # Skip header row
    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        title_element = cols[4].find_element(By.TAG_NAME, "a")
        title = title_element.get_attribute("title")
        titles.append(title)
    
    # Go to next page
    if page <= no_pages:
        next_button = driver.find_element(By.LINK_TEXT, "Next")
        next_button.click()
        time.sleep(2)  # Wait for page to load

# Close the WebDriver session
driver.quit()

In [12]:
data_list = []
driver = webdriver.Firefox()
for i in titles:
    details_url = f'https://nec.gov.np/applicant/detailview/{i}'
    driver.get(details_url)

    name_ind =          driver.find_element(By.XPATH, '/html/body/div[3]/div[1]/div/div[3]/div/div/section/div/table/tbody/tr[1]/td[1]').text
    gender_ind =        driver.find_element(By.XPATH, '/html/body/div[3]/div[1]/div/div[3]/div/div/section/div/table/tbody/tr[2]/td[1]').text
    passoutyear_ind =   driver.find_element(By.XPATH, '/html/body/div[3]/div[1]/div/div[3]/div/div/section/div/table/tbody/tr[4]/td[1]').text
    address_ind =       driver.find_element(By.XPATH, '/html/body/div[3]/div[1]/div/div[3]/div/div/section/div/table/tbody/tr[1]/td[2]').text
    uni_country =       driver.find_element(By.XPATH, '/html/body/div[3]/div[1]/div/div[3]/div/div/section/div/table/tbody/tr[3]/td[2]').text
    category      =     driver.find_element(By.XPATH, '/html/body/div[3]/div[1]/div/div[3]/div/div/section/div/table/tbody/tr[2]/td[2]').text
    qualification =     driver.find_element(By.XPATH, '/html/body/div[3]/div[1]/div/div[3]/div/div/section/div/table/tbody/tr[3]/td[1]').text
    reg_no=             driver.find_element(By.XPATH, '/html/body/div[3]/div[1]/div/div[3]/div/div/section/div/table/tbody/tr[4]/td[2]').text


    data = {
    "Name" : name_ind,
    "Gender": gender_ind,
    "Passout Year": passoutyear_ind,
    "University/Country": uni_country,
    "Category" : category,
    "Reg Number": reg_no,
    "Qualification":qualification,
    "Address":address_ind
}
    
    data_list.append(data)
driver.quit()    

In [14]:
df = pd.DataFrame(data_list)

In [16]:
df.to_csv("Post2020.csv", index=False)